<a href="https://colab.research.google.com/github/NikhileshSarangpure/Thyroid-classification-app/blob/main/thyroid_detection_using_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from  xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns



In [ ]:
allHyperTest = pd.read_csv("/content/allhyperTestEDIT.CSV")
allHyperTrain = pd.read_csv("/content/allhyperTrainEDIT.CSV")
allHypoTest = pd.read_csv("/content/allhypoTEST.csv")
allHypoTrain = pd.read_csv("/content/allhypoDATA.CSV")

display(allHypoTest.head(10))
display(allHypoTrain.dtypes)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target,ID
0,35,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,f,?,other,negative,219
1,63,M,f,f,f,f,f,f,f,f,...,108,t,0.96,t,113,f,?,SVI,negative,2059
2,25,F,f,f,f,f,f,f,f,f,...,61,t,0.82,t,75,f,?,SVHD,negative,399
3,53,F,f,f,f,f,f,f,f,t,...,145,t,1.03,t,141,f,?,other,negative,1911
4,92,F,f,f,f,f,f,f,f,f,...,120,t,0.84,t,143,f,?,SVI,negative,487
5,67,M,f,f,f,f,f,f,f,t,...,84,t,0.83,t,101,f,?,other,negative,1234
6,60,F,f,f,f,f,f,f,f,f,...,117,t,1.31,t,90,f,?,other,negative,1113
7,60,F,f,f,f,f,f,f,f,f,...,65,t,0.99,t,66,f,?,SVI,compensated_hypothyroid,1344
8,48,F,f,f,f,f,f,f,f,f,...,112,t,0.92,t,121,f,?,other,negative,2758
9,27,F,f,f,f,f,f,f,f,f,...,94,t,0.89,t,106,f,?,SVI,negative,3230


age                          object
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
sick                         object
pregnant                     object
thyroid_surgery              object
I131_treatment               object
query_hypothyroid            object
query_hyperthyroid           object
lithium                      object
goitre                       object
tumor                        object
hypopituitary                object
psych                        object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                         

In [ ]:
del allHyperTest["ID"]
del allHyperTrain["ID"]
del allHypoTest["ID"]
del allHypoTrain["ID"]

In [ ]:
def notCorrect_TargetFilter(df,correct_Target,target):
    df = df[df.Target.isin(correct_Target)]
    df.replace(correct_Target,target,inplace = True)
    return df
    
allHyperTest = notCorrect_TargetFilter(allHyperTest,["hyperthyroid","T3_toxic","goitre","secondary_toxic"],"hyperthyroid")
allHyperTrain = notCorrect_TargetFilter(allHyperTrain,["hyperthyroid","T3_toxic","goitre","secondary_toxic"],"hyperthyroid")
allHypoTest = notCorrect_TargetFilter(allHypoTest,["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")
allHypoTrain = notCorrect_TargetFilter(allHypoTrain,["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")

In [ ]:
allDataset = pd.concat([allHyperTest,allHyperTrain,allHypoTest,allHypoTrain], ignore_index = True)
display(allDataset.shape)

(393, 30)

In [ ]:
thyroid0387 = pd.read_csv("/content/thyroid0387EDIT.CSV")
display(thyroid0387.head(10))
display(thyroid0387.dtypes)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target,ID
0,29,F,f,f,f,f,f,f,f,t,...,?,f,?,f,?,f,?,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128,f,?,f,?,f,?,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,11,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,36,other,S,840803047
5,60,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,-,840803048
6,77,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,21,other,-,840803068
7,28,F,f,f,f,f,f,f,f,f,...,116,f,?,f,?,f,?,SVI,-,840807019
8,28,F,f,f,f,f,f,f,f,f,...,76,f,?,f,?,f,?,other,-,840808060
9,28,F,f,f,f,f,f,f,f,f,...,83,f,?,f,?,f,?,other,-,840808073


age                           int64
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
sick                         object
pregnant                     object
thyroid_surgery              object
I131_treatment               object
query_hypothyroid            object
query_hyperthyroid           object
lithium                      object
goitre                       object
tumor                        object
hypopituitary                object
psych                        object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                         

In [ ]:
del thyroid0387["ID"]

In [ ]:
thyroid0387['sex'] = thyroid0387['sex'].map({'F': 1, 'M': 0})

thyroid0387.replace(['A','B','C','D'],"hyperthyroid",inplace = True)
thyroid0387.replace(['E','F','G','H'],"hypothyroid",inplace = True)

for value in set(thyroid0387['Target']):
    if(value != 'hypothyroid' and value != 'hyperthyroid'):
        thyroid0387.replace(value,'negative',inplace=True)

In [ ]:
hypothyroid = pd.read_csv("/content/hypothyroid.csv")
display(hypothyroid.shape)
display(hypothyroid.head(10))
display(hypothyroid.dtypes)

(3163, 26)

,Unnamed: 0,Age,Sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,thyroid_surgery,query_hypothyroid,query_hyperthyroid,pregnant,...,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG
0,hypothyroid,72,M,f,f,f,f,f,f,f,...,y,0.6,y,15,y,1.48,y,10,n,?
1,hypothyroid,15,F,t,f,f,f,f,f,f,...,y,1.7,y,19,y,1.13,y,17,n,?
2,hypothyroid,24,M,f,f,f,f,f,f,f,...,y,0.2,y,4,y,1,y,0,n,?
3,hypothyroid,24,F,f,f,f,f,f,f,f,...,y,0.4,y,6,y,1.04,y,6,n,?
4,hypothyroid,77,M,f,f,f,f,f,f,f,...,y,1.2,y,57,y,1.28,y,44,n,?
5,hypothyroid,85,F,f,f,f,f,t,f,f,...,y,1.1,y,27,y,1.19,y,23,n,?
6,hypothyroid,64,F,f,f,f,t,f,f,f,...,y,1.3,y,54,y,0.86,y,63,n,?
7,hypothyroid,72,F,f,f,f,f,f,f,f,...,y,1.9,y,34,y,1.05,y,32,n,?
8,hypothyroid,20,F,f,f,f,f,t,f,f,...,n,?,y,39,y,1.21,y,32,n,?
9,hypothyroid,42,F,f,f,f,f,f,f,f,...,n,?,y,7.6,y,1.02,y,7.5,n,?


Unnamed: 0                   object
Age                          object
Sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
thyroid_surgery              object
query_hypothyroid            object
query_hyperthyroid           object
pregnant                     object
sick                         object
tumor                        object
lithium                      object
goitre                       object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                          object
dtype: object

In [ ]:
hypothyroid = hypothyroid.rename(columns={hypothyroid.columns[0]:"Target",hypothyroid.columns[1]:"age",hypothyroid.columns[2]:"sex" })
hypothyroid = hypothyroid[hypothyroid.Target.isin(['hypothyroid'])]

In [ ]:
sick_euthyroid = pd.read_csv("/content/sick-euthyroid.CSV")
display(sick_euthyroid.shape)
display(sick_euthyroid.head(10))
display(sick_euthyroid.dtypes)

(3163, 26)

,Target,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,thyroid_surgery,query_hypothyroid,query_hyperthyroid,pregnant,...,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG
0,sick-euthyroid,72,M,f,f,f,f,f,f,f,...,y,1,y,83,y,0.95,y,87,n,?
1,sick-euthyroid,45,F,f,f,f,f,f,f,f,...,y,1,y,82,y,0.73,y,112,n,?
2,sick-euthyroid,64,F,f,f,f,f,f,f,f,...,y,1,y,101,y,0.82,y,123,n,?
3,sick-euthyroid,56,M,f,f,f,f,f,f,f,...,y,0.80,y,76,y,0.77,y,99,n,?
4,sick-euthyroid,78,F,t,f,f,f,t,f,f,...,y,0.30,y,87,y,0.95,y,91,n,?
5,sick-euthyroid,80,M,f,f,f,f,f,f,f,...,y,0.80,y,105,y,0.88,y,120,n,?
6,sick-euthyroid,74,F,f,f,f,f,f,f,f,...,y,0.70,y,98,y,0.81,y,121,n,?
7,sick-euthyroid,?,F,f,f,f,f,f,f,f,...,y,1.10,y,121,y,1.11,y,109,n,?
8,sick-euthyroid,42,F,f,f,f,f,f,f,f,...,y,1.10,y,93,y,0.73,y,127,n,?
9,sick-euthyroid,89,M,f,f,f,f,f,f,f,...,y,0.80,y,111,y,0.68,y,165,n,?


Target                       object
age                          object
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
thyroid_surgery              object
query_hypothyroid            object
query_hyperthyroid           object
pregnant                     object
sick                         object
tumor                        object
lithium                      object
goitre                       object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                          object
dtype: object

In [ ]:
sick_euthyroid = sick_euthyroid[sick_euthyroid.Target.isin(['negative'])]
display(sick_euthyroid.shape)

(2870, 26)

In [ ]:
ann_train = pd.read_csv("/content/ann-train.CSV")
ann_test = pd.read_csv("/content/ann-test.CSV")
display(ann_test.head(10))
display(ann_test.dtypes)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,Target
0,0.29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00610,0.028,0.111,0.131,0.08500,2
1,0.32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00130,0.019,0.084,0.078,0.10700,3
2,0.35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00000,0.031,0.239,0.100,0.23900,3
3,0.21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00100,0.018,0.087,0.088,0.09900,3
4,0.22,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0.00040,0.022,0.134,0.135,0.09900,3
5,0.22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00160,0.020,0.123,0.113,0.10900,3
6,0.39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00160,0.036,0.133,0.144,0.09300,3
7,0.77,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00081,0.020,0.080,0.096,0.08316,3
8,0.23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00025,0.014,0.113,0.096,0.11746,3
9,0.23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00260,0.011,0.104,0.104,0.09900,3


age                          float64
sex                            int64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH                          float64
T3                           float64
TT4                          float64
T4U                          float64
FTI                          float64
Target                         int64
dtype: object

In [ ]:
target1 = pd.Series(ann_test[ann_test.columns[-1]].values)
display(target1.value_counts())
target2 = pd.Series(ann_train[ann_train.columns[-1]].values)
display(target2.value_counts())

3    3178
2     177
1      73
dtype: int64

3    3488
2     191
1      93
dtype: int64

In [ ]:
print("Sex thyroid0387 1=F,0=M:")
sex_series1 = pd.Series(thyroid0387[thyroid0387.columns[1]].values)
display(sex_series1.value_counts())
print("Sick-euthyroid:")
sex_series2 = pd.Series(sick_euthyroid[sick_euthyroid.columns[2]].values)
display(sex_series2.value_counts())

Sex thyroid0387 1=F,0=M:


1.0    6073
0.0    2792
dtype: int64

Sick-euthyroid:


F    2003
M     800
?      67
dtype: int64

In [ ]:
sex1 = pd.Series(ann_test[ann_test.columns[1]].values)
display(sex1.value_counts())
sex2 = pd.Series(ann_train[ann_train.columns[1]].values)
display(sex2.value_counts())

0    2380
1    1048
dtype: int64

0    2629
1    1143
dtype: int64

In [ ]:
ann = pd.concat([ann_train,ann_test], ignore_index = True)
ann['sex'] = ann['sex'].map({0:'F',1:'M'})
ann['Target'] = ann['Target'].map({3:'negative',2:'hypothyroid',1:'hyperthyroid'})

continuos_attributes = ['age','TSH','T3','TT4','T4U','FTI']
for attribute in continuos_attributes:
    ann[attribute] = ann[attribute] * 100

def fillNewAttributes(row,attribute):
    if row[attribute] > 0:
        return 'y'
    else:
        return 'n'

ann['TSH_measured'] = ann.apply(lambda row: fillNewAttributes(row,'TSH'), axis=1)
ann['T3_measured'] = ann.apply(lambda row: fillNewAttributes(row,'T3'), axis=1)
ann['TT4_measured'] = ann.apply(lambda row: fillNewAttributes(row,'TT4'), axis=1)
ann['T4U_measured'] = ann.apply(lambda row: fillNewAttributes(row,'T4U'), axis=1)
ann['FTI_measured'] = ann.apply(lambda row: fillNewAttributes(row,'FTI'), axis=1)
display(ann.dtypes)

age                          float64
sex                           object
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH                          float64
T3                           float64
TT4                          float64
T4U                          float64
FTI                          float64
Target                        object
TSH_measured                  object
T3_measured                   object
TT4_measured                  object
T4U_measured                  object
FTI_measured                  object
d

In [ ]:
data = pd.concat([allDataset,thyroid0387,hypothyroid,sick_euthyroid,ann], ignore_index = True)
display(data.shape)
display(data.dtypes)

(19786, 30)

age                          object
sex                          object
on_thyroxine                 object
query_on_thyroxine           object
on_antithyroid_medication    object
sick                         object
pregnant                     object
thyroid_surgery              object
I131_treatment               object
query_hypothyroid            object
query_hyperthyroid           object
lithium                      object
goitre                       object
tumor                        object
hypopituitary                object
psych                        object
TSH_measured                 object
TSH                          object
T3_measured                  object
T3                           object
TT4_measured                 object
TT4                          object
T4U_measured                 object
T4U                          object
FTI_measured                 object
FTI                          object
TBG_measured                 object
TBG                         

In [ ]:
data.loc[data.index.to_series().sample(frac=0.4).index, 'Hairloss'] = 1
data['Hairloss'].fillna(0,inplace=True)
data.loc[data.index.to_series().sample(frac=0.4).index, 'Constipation'] = 1
data['Constipation'].fillna(0,inplace=True)
data.loc[data.index.to_series().sample(frac=0.4).index, 'Nervousness'] = 1
data['Nervousness'].fillna(0,inplace=True)
data.loc[data.index.to_series().sample(frac=0.4).index, 'Skin'] = 1
data['Skin'].fillna(0,inplace=True)
data.loc[data.index.to_series().sample(frac=0.4).index, 'Feeling Tired'] = 1
data['Feeling Tired'].fillna(0,inplace=True)

In [ ]:
data=data.replace({"?":np.NAN})
data.isna().sum()

age                            409
sex                            394
on_thyroxine                     0
query_on_thyroxine               0
on_antithyroid_medication        0
sick                             0
pregnant                         0
thyroid_surgery                  0
I131_treatment                3021
query_hypothyroid                0
query_hyperthyroid               0
lithium                          0
goitre                           0
tumor                            0
hypopituitary                 3021
psych                         3021
TSH_measured                     0
TSH                           1321
T3_measured                      0
T3                            3372
TT4_measured                     0
TT4                            696
T4U_measured                     0
T4U                           1083
FTI_measured                     0
FTI                           1075
TBG_measured                  7200
TBG                          19174
referral_source     

In [ ]:
del data['TBG']
del data['referral_source']
del data['TBG_measured']
del data['sex']

In [ ]:
data.dropna(axis = 0, thresh = 20, inplace = True)
data.isna().sum()

age                           409
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment               3021
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                3021
psych                        3021
TSH_measured                    0
TSH                          1321
T3_measured                     0
T3                           3372
TT4_measured                    0
TT4                           696
T4U_measured                    0
T4U                          1083
FTI_measured                    0
FTI                          1075
Target                          0
Hairloss                        0
Constipation                    0
Nervousness                     0
Skin          

In [ ]:
data1 = data.interpolate(method = 'spline', order = 3)
display(data1.isna().sum())

age                           409
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment               3021
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                3021
psych                        3021
TSH_measured                    0
TSH                          1321
T3_measured                     0
T3                           3372
TT4_measured                    0
TT4                           696
T4U_measured                    0
T4U                          1083
FTI_measured                    0
FTI                          1075
Target                          0
Hairloss                        0
Constipation                    0
Nervousness                     0
Skin          

In [ ]:
data = data.replace({"t":1,"f":0, "y":1, "n":0, "hypothyroid":1, "negative":0,"hyperthyroid":2, "F":1, "M":0})
display(data.dtypes)

age                           object
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment               float64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                float64
psych                        float64
TSH_measured                   int64
TSH                           object
T3_measured                    int64
T3                            object
TT4_measured                   int64
TT4                           object
T4U_measured                   int64
T4U                           object
FTI_measured                   int64
FTI                           object
Target                         int64
Hairloss                     float64
C

In [ ]:
cols = data.columns[data.dtypes.eq('object')]
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')
display(data.dtypes)

age                          float64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment               float64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                float64
psych                        float64
TSH_measured                   int64
TSH                          float64
T3_measured                    int64
T3                           float64
TT4_measured                   int64
TT4                          float64
T4U_measured                   int64
T4U                          float64
FTI_measured                   int64
FTI                          float64
Target                         int64
Hairloss                     float64
C

In [ ]:
corr_values = abs(data[data.columns[0:]].corr()['Target'][:])
corr_values = corr_values.drop('Target')
corr_values = corr_values[corr_values > 0.04]
display(corr_values)

on_thyroxine          0.081920
query_hypothyroid     0.051018
query_hyperthyroid    0.074952
psych                 0.050080
TSH                   0.252202
TT4_measured          0.047733
FTI                   0.046017
Name: Target, dtype: float64

In [ ]:
def holdout(dataframe):
  x = dataframe[corr_values.index]
  y = dataframe['Target']
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42) 
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = holdout(data)

In [ ]:
classifiers = {
    "XGBClassifier" : XGBClassifier(learning_rate=0.01)}

In [ ]:
def classification(classifiers, X_train, X_test, y_train, y_test):
  res = pd.DataFrame(columns=["Classifier", 
                                "Accuracy", 
                                "Precision", 
                                "Recall", 
                                "F1Score"])
  for name, clf in classifiers.items():
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            pr, rc, fs, sup = metrics.precision_recall_fscore_support(y_test, y_pred, average='macro')
            res = res.append({"Classifier": name,"Accuracy": round(metrics.accuracy_score(y_test, y_pred), 4),
                              "Precision": round(pr, 4), "Recall":round(rc, 4), "F1Score":round(fs, 4)}, ignore_index=True)
            print("Confusion matrix for: ", name)
            display(confusion_matrix(y_test, y_pred))
  res.set_index("F1Score", inplace=True)
  res.sort_values(by="F1Score", ascending=False, inplace=True)   
  return res

display(classification(classifiers, X_train, X_test, y_train, y_test))

Confusion matrix for:  XGBClassifier


array([[5296,   50,    7],
       [ 170,  272,    2],
       [  91,    8,   40]])

,Classifier,Accuracy,Precision,Recall
F1Score,,,,
0.6997,XGBClassifier,0.9447,0.8645,0.6299


In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from pycaret.classification import *


In [ ]:
exp_clf101 = setup(data = data, target = 'Target', session_id=123)

,Description,Value
0,session_id,123
1,Target,Target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(19786, 31)"
5,Missing Values,True
6,Numeric Features,6
7,Categorical Features,24
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9762,0.9955,0.8999,0.9764,0.9762,0.8623,0.8625,0.619
gbc,Gradient Boosting Classifier,0.9753,0.9935,0.8752,0.9752,0.9750,0.8555,0.8559,5.426
rf,Random Forest Classifier,0.9736,0.9942,0.8599,0.9733,0.9732,0.8436,0.8440,1.292
dt,Decision Tree Classifier,0.9723,0.9250,0.8863,0.9729,0.9725,0.8410,0.8414,0.068
et,Extra Trees Classifier,0.9541,0.9827,0.7229,0.9518,0.9495,0.6829,0.7007,1.269
knn,K Neighbors Classifier,0.9258,0.8619,0.6004,0.9163,0.9180,0.4812,0.4959,0.249
lr,Logistic Regression,0.9199,0.8443,0.4519,0.9074,0.8982,0.3022,0.3736,6.165
lda,Linear Discriminant Analysis,0.9160,0.8427,0.5117,0.9040,0.8983,0.3235,0.3651,0.092
ridge,Ridge Classifier,0.9115,0.0000,0.3647,0.8840,0.8762,0.1169,0.2177,0.040
dummy,Dummy Classifier,0.9068,0.5000,0.3333,0.8223,0.8625,0.0000,0.0000,0.028


In [ ]:
exp_clf102 = setup(data = data, target = 'Target', session_id=123,
                  normalize = True, 
                  transformation = True)

,Description,Value
0,session_id,123
1,Target,Target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(19786, 31)"
5,Missing Values,True
6,Numeric Features,6
7,Categorical Features,24
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9773,0.9952,0.9101,0.9776,0.9773,0.8689,0.8691,0.604
rf,Random Forest Classifier,0.9751,0.9939,0.8654,0.9748,0.9746,0.8518,0.8523,1.282
gbc,Gradient Boosting Classifier,0.9747,0.9932,0.8747,0.9747,0.9744,0.8519,0.8523,5.381
dt,Decision Tree Classifier,0.9723,0.9250,0.8867,0.9729,0.9726,0.8415,0.8418,0.078
et,Extra Trees Classifier,0.9677,0.9875,0.8068,0.9664,0.9662,0.7983,0.8014,1.131
knn,K Neighbors Classifier,0.9462,0.9336,0.7149,0.9422,0.9429,0.6517,0.6581,0.780
lr,Logistic Regression,0.9380,0.9401,0.5635,0.9277,0.9287,0.5682,0.5844,1.482
svm,SVM - Linear Kernel,0.9360,0.0000,0.5181,0.9184,0.9222,0.5367,0.5626,0.179
lda,Linear Discriminant Analysis,0.9271,0.9192,0.6391,0.9274,0.9259,0.5838,0.5848,0.097
ada,Ada Boost Classifier,0.9095,0.8310,0.7365,0.9276,0.9159,0.5550,0.5645,0.545


In [ ]:
rf_model = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9711,0.9930,0.8505,0.9707,0.9706,0.8272,0.8277
1,0.9791,0.9956,0.8686,0.9791,0.9786,0.8760,0.8763
2,0.9776,0.9959,0.9093,0.9777,0.9777,0.8704,0.8705
3,0.9733,0.9890,0.8817,0.9731,0.9731,0.8425,0.8426
4,0.9718,0.9935,0.8366,0.9711,0.9711,0.8303,0.8311
5,0.9755,0.9921,0.8804,0.9751,0.9752,0.8543,0.8546
6,0.9776,0.9947,0.8547,0.9775,0.9769,0.8655,0.8663
7,0.9726,0.9939,0.8464,0.9718,0.9720,0.8371,0.8374
8,0.9798,0.9964,0.8981,0.9798,0.9796,0.8817,0.8818


In [ ]:
evaluate_model(rf_model)

In [ ]:
predict_model(rf_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.9727,0.9932,0.8789,0.9724,0.9725,0.8414,0.8416


,age,TSH,T3,TT4,T4U,FTI,on_thyroxine_0,query_on_thyroxine_1,on_antithyroid_medication_0,sick_0,...,TT4_measured_1,FTI_measured_0,Hairloss_0.0,Constipation_1.0,Nervousness_0.0,Skin_1.0,Feeling Tired_1.0,Target,Label,Score
0,-0.480646,-0.909742,1.785951,-1.032803,1.745054,-1.185577,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,1.00
1,-1.398523,-0.754022,0.222524,-1.140728,1.286964,-1.156651,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,1.00
2,1.028481,1.235571,0.126697,1.167155,-0.582601,0.454662,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0.95
3,-0.702362,-0.218187,0.655024,0.618730,-0.836190,0.651134,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,0.99
4,0.935862,-0.720465,-0.157186,-1.115656,1.133449,-1.080726,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5931,0.406737,-0.786260,-0.157186,-1.058026,1.400171,-1.109158,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.98
5932,1.165412,-0.616299,0.249750,-1.152759,1.335924,-1.183176,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,1.00
5933,0.406737,2.404971,0.139210,-1.166717,-0.758902,-1.221697,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,0,0.81
5934,1.074386,-0.738429,0.153273,-1.037383,1.286964,-1.047711,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0,1.00


In [ ]:
save_model(rf_model, model_name = 'random-forest_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Target',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strat...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None, max_features='auto',
                                         max_l

In [ ]:
from google.colab import files
files.download('random-forest_model.pkl') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
classifiers1 = {
    "Random Forest": RandomForestClassifier(),
    "K-Nearest Neighbors" : KNeighborsClassifier(4),
    "Decision Tree" : DecisionTreeClassifier(),
    "Naive Bayes" : GaussianNB(),
    "SVM" : SVC()
}

In [ ]:
corr_values = abs(data1[data1.columns[0:]].corr()['Target'][:])
corr_values = corr_values.drop('Target')
corr_values = corr_values[corr_values > 0.04]
display(corr_values)

X_train1, X_test1, y_train1, y_test1 = holdout(data1)

display(classification(classifiers1,X_train1, X_test1, y_train1, y_test1))